<a href="https://colab.research.google.com/github/lail-lei/nlp-cupcakes/blob/main/fast_keyword_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
import math
import re
from textblob import TextBlob as tb
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from google.colab import files

import io

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [101]:
# our data from csv
df = pd.read_csv('https://raw.githubusercontent.com/lail-lei/nlp-cupcakes/main/csvs/cupcakes_with_frosting.csv')

In [102]:
reduced_df = df.iloc[:, 0:6]
reduced_df

,Title,Yield,Ingredients,Steps,URL,Tags
0,Coconut Cupcakes,18 to 20 cupcakes,[{'frosting': ['1 pound cream cheese at room t...,['Preheat the oven to 325 degrees F. In the bo...,https://www.foodnetwork.com/recipes/ina-garten...,"['Baking', 'Dessert', 'Mixer Recipes', 'Cupcak..."
1,Go-To Vanilla Cupcakes,12 cupcakes (or about 48 mini cupcakes),"[{'main': ['1 1/2 cups all purpose flour', '1 ...",['Preheat the oven to 350 degrees F and positi...,https://www.foodnetwork.com/recipes/food-netwo...,"['Baking', 'Dessert', 'Cupcake']"
2,Vanilla Cupcakes,12 cupcakes,"[{'frosting': ['4 large egg whites', '3/4 cup ...",['Preheat the oven to 350 degrees F. Line a 12...,https://www.foodnetwork.com/recipes/food-netwo...,"['Easy Baking', 'Cupcake']"
3,Magnolia's Vanilla Cupcake,24 cupcakes,"[{'main': ['1 1/2 cups self rising flour', '1 ...",['Preheat oven to 350 degrees F. Line 2 muffin...,https://www.foodnetwork.com/recipes/magnolias-...,"['Easy Dessert Recipes', 'Dessert', 'Easy Baki..."
4,Lemon Lime Cupcakes,48 mini cupcakes,"[{'main': ['1 3/4 cups all purpose flour', '1/...",['For the batter Preheat the oven to 350 degre...,https://www.foodnetwork.com/recipes/ree-drummo...,"['Cupcake', 'Dessert']"
...,...,...,...,...,...,...
750,Buttercream by Billington's,12,"[{'main': ['50g Unsalted butter ', '200g Billi...",['Beat the butter and sugar together until smo...,https://www.bakingmad.com/recipes/basic-butter...,NaN
751,Red Velvet Cake,9,"[{'main': ['120g Unsalted butter', '300g Silve...","['Preheat your oven to 180 C .', 'Line 2 x 20c...",https://www.bakingmad.com/recipes/red-velvet-cake,NaN
752,Whoopie Pies by Lorraine Pascale,6,"[{'main': ['120ml Milk ', '190g Unrefined deme...","['Preheat the oven to 170 C .', 'Line 2 baking...",https://www.bakingmad.com/recipes/whoopie-pies,NaN
753,Toffee Sauce,40,"[{'sauce': ['50ml Water', '300g Billington s U...",['To make the sauce use a large heavy bottomed...,https://www.bakingmad.com/recipes/toffee-sauce,NaN


In [103]:
def process (text):
  all_stopwords = stopwords.words('english')
 
  recipe_specific = ["title", "recipe","stick", "www", "com", "http", "tag", "mixer", "oven", "liner", "halfway", "minute", "step", "tag", "tester", "preheat", "rack", "pan", "tin", "mini", "bowl", "medium", "large", "cm", "small", "ounce", "gina", "betty crocker", "baking", "temperature", "according", "mug", "mix", "nielsen massey", "nielsen", "massey", "beat", "whisk", "set", "aside", "every", "each", "main", "whisk", "room", "set aside", "unnamed", "beat", "ingredient", "ingredients", "amount", "unit", "type", "text", "teaspoon", "teaspoons", "tsp", "cup", "cups", "tablespoon", "tablespoons", "tbsp", "tb", 
                     "grams", "gram", "g", "gm", "gms", "pounds", "lbs", "pounds", "pound", "kilogram", "kg", "kilograms", "kgs", "kilo", "kilos", "milliliter", "ml", 
                     "pinch", "pinches", "dash", "dashes", "NaN", "null", "steps", "tags", "description", "f", "c", "yield", "tin", "muffin tin", "billington", "nielson massey", "allinson"]
  all_stopwords.extend(recipe_specific)
  text = re.sub(r"[^a-zA-Z]+", ' ', text).lower().strip()
  tokens = word_tokenize(text)
  lemmatizer = WordNetLemmatizer()
  lemmatized = [lemmatizer.lemmatize(w) for w in tokens]
  removing_custom_words = [words for words in lemmatized if not words in all_stopwords]
  pos_tagged = nltk.pos_tag(removing_custom_words)
  # remove all words that are not nouns or adjectives
  nouns = list(filter(lambda x:x[1]=='NN',pos_tagged))
  adj = list(filter(lambda x:x[1]=='JJ',pos_tagged))
  nouns = [x[0] for x in nouns]
  adj = [x[0] for x in adj]
  return " ".join([*nouns, *adj])

In [104]:
reduced_df["processed"] = reduced_df.apply(lambda x: process(x.to_json()), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [105]:
reduced_df["processed"][1]

'vanilla cupcake cupcake powder fine salt egg sugar butter vanilla extract milk frosting position line standard muffin muffin powder salt egg sugar light butter vanilla mixture milk mixture care batter batter cake center cupcake time cupcake cupcake cool decorate foodnetwork food network kitchen cupcake flour unsalted pure favorite degree flour electric add half flour flour come clean standard cool frost url vanilla'

In [108]:
docs = reduced_df["processed"]

In [109]:
# sklearn's tfidf vectorizer to create matrix
tfIdfVectorizer = TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(docs)

# array to hold keywords
keywords = []

# get keywords for each document
for i in range(755):
  matrix = pd.DataFrame(tfIdf[i].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
  matrix = matrix.sort_values('TF-IDF', ascending=False)
  matrix = matrix[:5]
  keywords.append([*matrix.index])
 

In [110]:
reduced_df["keywords"] = keywords

In [111]:
reduced_df.to_csv('cupcakes_with_frosting_keywords.csv') 
files.download('cupcakes_with_frosting_keywords.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>